In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
import json
import random
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/chatbot/chatbot_model.h5')

In [4]:
intents = json.loads(open('/content/drive/MyDrive/Colab Notebooks/chatbot/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/chatbot/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/chatbot/classes.pkl','rb'))

In [5]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [9]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [16]:
p = bow('who are you', words,show_details=False)

In [17]:
res = model.predict(np.array([p]))

1/1 [==============================] - 0s 24ms/step


In [18]:
res

array([[7.92244555e-06, 1.94162440e-05, 1.42153571e-04, 2.56836007e-04,
        4.20606884e-05, 2.20800997e-04, 7.73859429e-05, 1.43395830e-03,
        1.59398678e-05, 2.64046389e-06, 2.72441266e-05, 3.07577357e-05,
        6.69416806e-07, 6.68249559e-05, 8.62350953e-06, 1.82668373e-05,
        5.23632989e-05, 1.15939845e-04, 1.46583297e-05, 3.29371760e-05,
        1.29415075e-05, 4.89433432e-06, 7.25506034e-05, 1.31658511e-04,
        8.22691596e-04, 5.07368604e-05, 1.61091087e-03, 2.70075677e-03,
        6.80168741e-05, 2.80707056e-04, 1.11370782e-05, 7.55247675e-05,
        5.40240171e-06, 5.52505735e-05, 3.64476640e-04, 5.80531657e-01,
        6.29530987e-05, 6.80880694e-05, 4.77607900e-05, 8.12948347e-05,
        3.79391364e-04, 1.03165512e-05, 3.25745605e-05, 4.96898938e-05,
        3.62371634e-06, 4.34960471e-03, 2.31056125e-03, 9.96661372e-04,
        2.21359500e-04, 3.91510657e-05, 2.13372451e-03, 1.67713697e-05,
        2.67199415e-04, 3.98998141e-01, 5.44360024e-04]], dtype=

In [24]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [25]:
predict_class('who are you?', model)

1/1 [==============================] - 0s 27ms/step
[[35, 0.58053166], [53, 0.39899814]]


[{'intent': 'name1', 'probability': '0.58053166'},
 {'intent': 'who', 'probability': '0.39899814'}]

In [20]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [21]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [29]:
chatbot_response('who are you')

1/1 [==============================] - 0s 43ms/step
[[35, 0.58053166], [53, 0.39899814]]


"I'm a KCA chat agent"